# Project : Analysis of the topics covered by Italian politicians on Twitter during Covid19 pandemic

In [ ]:
import inspect
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
from scipy.sparse import lil_matrix
import itertools
import random
from scipy import sparse as sp
%pylab inline
%load_ext autoreload
%autoreload 2

## Data loading and preparation

In [ ]:
# Lettura dei dati 
dati = pd.read_csv("tantitweet9.csv", encoding="utf-8", low_memory=False)
print("Dimensione del dataset: {} x {}".format(*dati.shape))

In [ ]:
# Salvo il nome dei politici in una lista
nome_politici = dati["screen_name"].unique()
print(nome_politici)

In [ ]:
# creo una lista contenente una lista per ogni politico dei suo tweet
tweet = []
for nome in (nome_politici):
        tweet.append(dati.loc[dati["screen_name"] == nome]["text"])

In [ ]:
# filtro i tweet che sono piu lunghi di 200 caratteri
# per ogni tweet associo l'autore e lo salvo in una lista
tweet2 = []
autore = []
for i,t in enumerate(tweet):
    lista_temp = []
    for k in t:
        if len(k) > 200:
            lista_temp.append(k)
            autore.append(nome_politici[i])
    tweet2.append(lista_temp)

In [ ]:
# numero tweet per politico
dim = []
for i,t in enumerate(tweet2):
    print(i,len(t), nome_politici[i]) 
    dim.append(len(t))

# Pre-processing

In [ ]:
# Librerie utilizzate per il pre-processing
from bs4 import BeautifulSoup
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from string import punctuation
import tqdm
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
# Funzione che esegue le operazioni di pre-processing
def tweet_analyzer(tweet, tokenizer, stop_words):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub("http\S+", "", tweet)
    tweet = tweet.lower()
    tweet = tokenizer.tokenize(tweet)
    tweet = [token for token in tweet if token not in stop_words
        and not token.isdigit()]
    tweet = [token for token in tweet if len(token) > 1]
    return tweet

In [ ]:
# liste di stopword scaricate da nltk
nltk.download("stopwords")
nltk.download("wordnet")
# lista di stopword contenti nomi e partiti dei politici e altre parole senza significato
piu_stopwords = ["oggi", "ora","solo","cosa","giuseppeconteit", "fatto", "quando", "poi", "dire","essere","mai","prima","molto","albertobagnai","borghi_claudio",
                "sempre","modo","punto","nulla","giorni","deve","così","può","cose","fa","anni","dobbiamo","the","alberto","bagnai","magari","ecco","gasparripdl",
                 "and","dare","due","tutte","insieme","berlusconi","l'italia","va","bisogna","meno","parte","ce","grande","oggi","fare","senza","quindi",
                "sempre", "grazie", "ancora", "momento", "ogni", "dopo", "già", "sempre", "salvini","ieri","altro","pd","lega","m5s","forza_italia",
                "borghi_claudio","italiaviva","matteosalvinimi","bonafede","teresabellanova","rinaldi_euro","giorgio_gori","civati","matteorenzi","sbonaccini",
                "boccia","pure","invece","ore","tanti","qui","fra","pdnetwork","nzingaretti","assembleapd","zappingradio1","lauraboldrini","sinistra","destra",
                "laura","boldirni","tutta","qualcuno","emmabonino","piueuropalive","emma","bonino","piu_europa","vedova","simona","luigidimaio","centrodestra",
                "roberto_fico","ettore_rosato","elenabonetti","ferra2113","robersperanza","viva","ep_president","europa_it","annaascani","link","serve",
                "azione_it","carlocalenda","fdi","fratelliditalia","giorgiameloni","zaiapresidente","marattin","fratelliditaiia","lucianonobili",
                "marcocappa_","silviaromano","pennacchiiiii","chiaralessi","rinaldi","antonio","paologentiloni","gentiloni","ass_coscioni","f_boccia",
                "marcocappato","filomena_gallo","antonio_tajani","de","el","en","msgelmini","enricoletta","de","et","les","st","mara_carfagna","bene",
                "far","pfmajorino","sbonaccini","bendellavedova","opencamera","laveritaweb","laveritaallesette","capezzone","alessiamorani","marco",
                 "renatobrunetta","giornatadellaterra","scorrettissima","politicamente","politicamente_scorrettissima","peoplepubit","gualtierieurope",
                "renzi","sa","troppo","pare","volete"]
stopwords2 = ['ad','al','allo','ai','agli','all','agl','alla','alle','con','col','coi','da','dal','dallo','dai','dagli','dalla','dalle','di','del','dello','dei','degli','della','delle','in','nel','nello','nei','negli','nell','negl','nella','nelle','su','sul','sullo','sui','sugli','sull','sugl','sulla','sulle','per','tra','contro','io','tu','lui','lei','noi','voi','loro','mio','mia','miei','mie','tuo','tua','tuoi','tue','suo','sua','suoi','sue','nostro','nostra','nostri','nostre','vostro','vostra','vostri','vostre','mi','ti','ci','vi','lo','la','li','le','gli','ne','il','un','uno','una','ma','ed','se','perché','anche','come','dove','che','chi','cui','non','più','quale','quanto','quanti','quanta','quante','quello','quelli','quella','quelle','questo','questi','questa','queste','si','tutto','tutti','ho','hai','ha','abbiamo','avete','hanno','abbia','abbiate','abbiano','avrò','avrai','avrà','avremo','avrete','avranno','avrei','avresti','avrebbe','avremmo','avreste','avrebbero','avevo','avevi','aveva','avevamo','avevate','avevano','ebbi','avesti','ebbe','avemmo','aveste','ebbero','avessi','avesse','avessimo','avessero','avendo','avuto','avuta','avuti','avute','sono','sei','è','siamo','siete','sia','siate','siano','sarò','sarai','sarà','saremo','sarete','saranno','sarei','saresti','sarebbe','saremmo','sareste','sarebbero','ero','eri','era','eravamo','eravate','erano','fui','fosti','fu','fummo','foste','furono','fossi','fosse','fossimo','fossero','essendo','faccio','fai','facciamo','fanno','faccia','facciate','facciano','farò','farai','farà','faremo','farete','faranno','farei','faresti','farebbe','faremmo','fareste','farebbero','facevo','facevi','faceva','facevamo','facevate','facevano','feci','facesti','fece','facemmo','faceste','fecero','facessi','facesse','facessimo','facessero','facendo','sto','stai','sta','stiamo','stanno','stia','stiano','starò','starai','starà','staremo','starete','staranno','starei','staresti','starebbe','staremmo','stareste','starebbero','stavo','stavi','stava','stavamo','stavate','stavano','stetti','stesti','stette','stemmo','steste','stettero','stessi','stesse','stessimo','stessero','stando']
LATIN_1_CHARS = [ "'",'e','-','-','-','-','-','-', "'","'",'"','"', '"','"','"','...',"'", "'", "'", "'", "'","'","+","-","=","(",")","’","‘","“","”", "»","«",""]

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('italian') + stopwords.words('english') + list(punctuation) + LATIN_1_CHARS + piu_stopwords + stopwords2

In [ ]:
# creo la lista dei token dei tweet -- una sottolista per ogni politico
token = []
for t in tweet2:
    token.append([tweet_analyzer(k, tokenizer, stop_words) for k in t])

In [ ]:
token2 = []
for i in range(0,len(token)):
    token2 = token2 + list(token[i])

# Descriptive analysis

In [ ]:
from collections import Counter

In [ ]:
#token per il grafico per il singolo politico
token4 = []
for i in range(0,len(token)):
    token4.append(list(itertools.chain.from_iterable(token[i])))

In [ ]:
from wordcloud import WordCloud

In [ ]:
token_salvini=token4[17]
token_renzi=token4[18]
token_gasparri=token4[24]
token_boldrini=token4[3]
token_civati=token4[28]
token_calenda=token4[19]
token_renzi=token4[18]

In [ ]:
datogliere=["italia","italiani","governo","paese","stato","italiano","italiana","serve","altri","gasparri","gruppofisenato","boldrini","senato",
            "forzaitalia","senatostampa"]
token_salvini=[elem for elem in token_salvini if elem not in datogliere]
token_gasparri=[elem for elem in token_gasparri if elem not in datogliere]
token_civati=[elem for elem in token_civati if elem not in datogliere]
token_calenda=[elem for elem in token_calenda if elem not in datogliere]
token_renzi=[elem for elem in token_renzi if elem not in datogliere]
token_boldrini=[elem for elem in token_boldrini if elem not in datogliere]

In [ ]:
c_salvini=Counter(token_salvini)
c_gasparri=Counter(token_gasparri)
c_boldrini=Counter(token_boldrini)
c_civati=Counter(token_civati)
c_renzi=Counter(token_renzi)
c_calenda=Counter(token_calenda)


In [ ]:
## Per i colori delle parole nella wordcloud
def random_color_func_dx(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    h = int(360.0 * 80 / 255.0)
    s = int(100.0 * 255.0 / 255.0) #verde
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(h, s, l)
def random_color_func_sx(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    h = int(360.0 * 2 / 255.0)
    s = int(100.0 * 255.0 / 255.0) #rosso
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(h, s, l)
def random_color_func_centrodx(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    h = int(360.0 * 150 / 255.0)
    s = int(100.0 * 255 / 255.0) #blu
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(h,s, l)

def random_color_func_rosa(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    h = int(360.0 * 0 / 255.0)
    s = int(100.0 * 255 / 255.0) #rosa
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(300,100, 50)

In [ ]:
MASK = plt.imread("figures/UntitledBlackCloud.jpg")
MAX_WORDS = 200
MAX_FONT_SIZE = 100
RELATIVE_SCALING = 0.1

plt.show()
plt.imshow(MASK)

wc_tutti = WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
).generate_from_frequencies(c_token)


wc_salvini= WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
    color_func=random_color_func_dx,
).generate_from_frequencies(c_salvini)

wc_gasparri= WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
    color_func=random_color_func_centrodx,
).generate_from_frequencies(c_gasparri)


wc_civati = WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
    color_func=random_color_func_sx,
).generate_from_frequencies(c_civati)

wc_boldrini = WordCloud(
    mask=MASK,
    max_words=MAX_WORDS, 
    background_color="white",
    max_font_size=MAX_FONT_SIZE,
    relative_scaling=RELATIVE_SCALING,
    color_func=random_color_func_rosa,
).generate_from_frequencies(c_boldrini)



In [ ]:
###word cloud per tutti
plt.figure(figsize=(10, 20))
plt.imshow(wc_tutti, interpolation='bilinear')
plt.axis("off")

In [ ]:
## wordcloud salvini
plt.figure(figsize=(10, 20))
plt.imshow(wc_salvini,interpolation="bilinear")
plt.axis("off")

In [ ]:
#word cloud gasparri
plt.figure(figsize=(10, 20))
plt.imshow(wc_gasparri,interpolation="bilinear")
plt.axis("off")

In [ ]:
#wordcloud boldrini
plt.figure(figsize=(10, 20))
plt.imshow(wc_boldrini,interpolation="bilinear")
plt.axis("off")

In [ ]:
#wordcloud civati
plt.figure(figsize=(10, 20))
plt.imshow(wc_civati,interpolation="bilinear")
plt.axis("off")

# LDA fitting

In [ ]:
import gensim #conda install gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
import time
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [ ]:
# creo dei token formati da due parole 
from gensim.models import Phrases

bigram = Phrases(token2, min_count=10)
for idx in range(len(token2)):
    for t in bigram[token2[idx]]:
        if '_' in t:
            token2[idx].append(t)

In [ ]:
id2word = corpora.Dictionary(token2)
id2word.filter_extremes( no_below=10)
corpus = [id2word.doc2bow(text) for text in token2]

In [ ]:
print('Number of unique tokens: %d' % len(id2word))
print('Number of documents: %d' % len(corpus))

In [ ]:
inizio = time.time()
num_topics = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           passes=20,
                                           random_state = 142
                                           )
fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

### LDA interpretation

In [ ]:
# Distribuzione delle parole nei topic
for i in range(0,num_topics):
    print("Topic: ",i)
    print(lda_model.show_topic(i,6))

#### pyLDAvis
Funzione per l'intepretazione dei topic
https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf

In [ ]:
import pyLDAvis # pip install pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Visualizare i topic per l'interpretazione
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
# Assegno ad ogni topic un titolo
topic_label = {'0':'terapia intensiva','1':'covid19 - lombardia','2':'palazzo chigi','3':'paese e solidarietà','4':'aiuto alle imprese e alle famiglie',
               '5':'volontari / finanza','6':'emergenza covid19','7':'economia (pil, miliardi, euro, ...)','8':'non definito','9':'ricorrenze',
               '10':'crisi del lavoro','11':'governo','12':'politica','13':'contagi e tamponi','14':'personale sanitario','15':'conferenza stampa',
               '16':'mes e europa','17':'lavoro - cassa integrazione','18':'spesa-crescita / scuola','19':'emergenza (sanitaria, economica)'}

### Topics frequencies

In [ ]:
# creo un dataframe con: id_tweet, autore, topic dominante, 
topics_df = pd.DataFrame()
for i, row_list in enumerate(lda_model[corpus]):
    row = row_list[0] if lda_model.per_word_topics else row_list
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
         if (j == 0):
            topics_df = topics_df.append(pd.Series([autore[i], topic_label[str(topic_num)]]), ignore_index=True)        
         else:
             break
topics_df.columns = ['Autore', 'Dominant_Topic']

In [ ]:
c_topic_politico = []
for i,nome in enumerate(nome_politici):
    c_topic_politico.append(Counter(topics_df.loc[topics_df["Autore"] == nome]["Dominant_Topic"]))

In [ ]:
c_topic = Counter(topics_df["Dominant_Topic"])

In [ ]:
# grafico frequenza complessiva dei topic nei documenti
N = num_topics

plt.figure(figsize=(30, 6))

plt.subplot(121)
plt.title("Frequenza complessiva dei topic nei documenti", fontsize='x-large', fontweight=1000)
plt.barh(*zip(*c_topic.most_common(N)), color="gold")
plt.yticks(fontsize = "x-large")
plt.xticks(fontsize = "x-large")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# grafico frequenza dei topic per singolo politico
N = num_topics
id_politico = 0
plt.figure(figsize=(10, 6))

plt.subplot(121)
plt.title("Topics piu frequenti di {}".format(nome_politici[id_politico]),fontsize='x-large', fontweight=1000)
plt.barh(*zip(*c_topic_politico[id_politico].most_common(5)), color="gold")
plt.yticks(fontsize = "x-large")
plt.xticks(fontsize = "x-large")
plt.gca().invert_yaxis()

plt.show()

In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=142,
                                           #chunksize=100,
                                           passes=20,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=token2, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 20
max_topics = 60
step_size = 2
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               #gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = [ '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=20)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            #for a in alpha:
                # iterare through beta values
                #for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, k=k, a='auto', b='auto')
                    # Save the model results
                    #model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    #model_results['Alpha'].append(a)
                    #model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()